In [1]:
# 链接:https://pan.baidu.com/s/1YI9gJSeJS97_-NDpJ-sN7g  密码:fcj6
import numpy as np 
import pandas as pd 
import shutil as sh
import os
from tqdm.auto import tqdm

In [3]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [4]:
df.head(2)

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0


In [5]:
index = list(set(df.image_id))

In [7]:
source = 'train'
if True:
    for fold in [0,1,2,3,4]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in df.groupby('image_id'):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [ ]:
!python train.py --img 512 --batch 2 --epochs 100 --data ./data/wheat0.yaml --cfg ./models/yolov5x.yaml --name yolov5x_fold0 --weights ./weights/yolov5x.pt

In [ ]:
!python train.py --img 512 --batch 2 --epochs 100 --data ./gwdconfig/wheat1.yaml --cfg ./gwdconfig/yolov5x.yaml --name yolov5x_fold1 --weights ./weights/yolov5x.pt

In [ ]:
!python train.py --img 512 --batch 2 --epochs 100 --data ./gwdconfig/wheat2.yaml --cfg ./gwdconfig/yolov5x.yaml --name yolov5x_fold2 --weights ./weights/yolov5x.pt

In [ ]:
!python train.py --img 512 --batch 2 --epochs 100 --data ./gwdconfig/wheat3.yaml --cfg ./gwdconfig/yolov5x.yaml --name yolov5x_fold3 --weights ./weights/yolov5x.pt

In [ ]:
!python train.py --img 512 --batch 2 --epochs 100 --data ./gwdconfig/wheat4.yaml --cfg ./gwdconfig/yolov5x.yaml --name yolov5x_fold4 --weights ./weights/yolov5x.pt

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 7 (delta 5), reused 5 (delta 5), pack-reused 2
Unpacking objects: 100% (7/7), done.
From https://github.com/ultralytics/yolov5
   37e13f8..0bc80e1  master     -> origin/master
Your branch is behind 'origin/master' by 17 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Namespace(adam=False, batch_size=2, bucket='', cache_images=False, cfg='./gwdconfig/yolov5x.yaml', data='./gwdconfig/wheat4.yaml', device='', epochs=100, evolve=False, img_size=[1024], multi_scale=False, name='yolov5x_fold4', noautoa

Dockerfile    detect.py          results.png                train_batch0.jpg
Kaggle.ipynb  gwdconfig/         results_yolov5x_fold0.txt  train_batch1.jpg
LICENSE       hubconf.py         runs/                      train_batch2.jpg
README.md*    inference/         test.py                    tutorial.ipynb
__pycache__/  labels.png         test_batch0_gt.jpg         utils/
convertor/    models/            test_batch0_pred.jpg       weights/
data/         requirements.txt*  train.py
